In [2]:
#Lisa koodi normaliseerimise definitsioonid
import re
import os
import psycopg2 as pg2
import psycopg2.extras
from psycopg2 import sql
import sys
import pandas as pd
sys.path.append('/home/anneott/Repos/cda-data-cleaning')
from configuration import database_connection_string
from common.db_operations import insert_to_database

In [3]:
def normalization(k):
        if k:
            k = re.sub('_+', ' ', k)
            if k.startswith(('E-', 'e-')):
                k = k[2:]
            #kui k algab numbriga, siis kustutab alguses oleva(d) numbri(d)
            if not k.isdigit() and k[0].isdigit() and not k.startswith('10h'):
                k = re.sub(r'\d+(.*)', '\\1', k)
            if re.search(r'(-%)|(-arv)', k):
                k = k.replace('-%', '%')
                k = k.replace('-arv', '#')
            if k.startswith('#'):
                k = k[1:] + '#'
            if k.startswith('%'):
                k = k[1:] + '%'
        return k.lower() #.replace(" ", "")


In [4]:
def get_half_normalized_code(code):
    code = re.sub('_+', ' ', code)
    if code.startswith(('E-', 'e-')):
        code = code[2:]
    if not code.isdigit() and code[0].isdigit() and not code.startswith('10h'):
        code = re.sub(r'\d+(.*)', '\\1', code)
    if re.search(r'(-%)|(-arv)', code):
        code = code.replace('-%', '%')
        code = code.replace('-arv', '#')
    return code

def get_normalized_code(code):
    if code:
        code = get_half_normalized_code(code)
        if code.startswith('#'):
            code = code[1:] + '#'
        if code.startswith('%'):
            code = code[1:] + '%'
        return code.lower().replace(" ", "")
    return ''

def get_full_words(analysis_string):
    if 'keskm' in analysis_string and not 'keskmine' in analysis_string:
        analysis_string = re.sub('keskm(\.*)','keskmine', analysis_string)
    return analysis_string

#annan sisse 2 korda sama asja? siis returnib [sulgude väline osa, sulgude sisene osa]
def get_content_parantheses(analysis, code, type_d='nimetus', part='code'):
    first = re.search(r'(.*)\((.*)\)', get_normalized_code(analysis)).group(1)
    second = re.search(r'(.*)\((.*)\)', code).group(2)
    if first != second.lower().replace(" ", ""):
        second = get_full_words(second)
        if re.search(r'(protsent|abs|arv|#|%)', second) or re.search(r'(protsent|abs|arv|#|%)', first):
            first, second = analysis_percent_absolut_values(first, second)
    second = second.lower().replace(" ", "")
    if type_d == 'kas':
        return [first, second]
    else:
        return [first, ''.join(e for e in second.lower() if e.isalnum())]


In [300]:
b = 'U-Bil_____strip'
c = re.sub('_+', ' ', b)
print(c)

U-Bil strip


In [4]:
#get_content_parantheses('456(123)gram', '456(123)gram')
analysis = 'hem(o)gramm'
code = 'baso(arv)'
type_d='nimetus'
part='code'

first = re.search(r'(.*)\((.*)\)', get_normalized_code(analysis)).group(1)
second = re.search(r'(.*)\((.*)\)', code).group(2)
print('first:', first, '; second: ', second)
if first != second.lower().replace(" ", ""):
    second = get_full_words(second)
    print('1')
    if re.search(r'(protsent|abs|arv|#|%)', second) or re.search(r'(protsent|abs|arv|#|%)', first):
        print('2')
        first, second = analysis_percent_absolut_values(first, second)
        print(first, second)
second = second.lower().replace(" ", "")
print(second)
if type_d == 'kas':
    print([first, second])
else:
    print([first, ''.join(e for e in second.lower() if e.isalnum())])


first: hem ; second:  arv
1
2


NameError: name 'analysis_percent_absolut_values' is not defined

In [5]:
#see eelmine müstiline funkstioon kasutab seda
def analysis_percent_absolut_values(f, s):
    if s.find('protsent'): #muudab sõna protsent sõnaks suhtarv; kui sõna on s alguses, siis ei leia seda
        s = re.sub('protsent', 'suhtarv', s)
    if s.find('%'):
        s = re.sub('%', 'suhtarv', s)
    if s.find('absoluutarv'):
        s = re.sub('absoluutarv', 'arv', s)
    if s.find('abs'): 
        s = re.sub('abs(.)?', 'arv', s) #mõnikord kaotab ära esimese sulu,teine sulg jääb alles nt cabs(1) -> carv1)
    if 'suhtarv' in s and "%" not in f:
        f = f + "%"
    if 'arv' in s and 'suhtarv' not in s and "#" not in f:
        f  = f + '#'
    if '#' in s:
        if '#' not in f:
            f = f + '#'
        s = s.replace('#', '') + "#"
    if f.startswith('%'):
        f = f[1:] + "%"
    if f.startswith('#'):
        f = f[1:] + "#"
    return f, s

In [6]:
symbols_to_replace = {
        'ü': 'u',
        'ä': 'a',
        'Ü': 'U',
        'Ä': 'A',
        'Ō': 'O',
        'õ': 'o'
    }

def sanitize_string(s):
    if s:
        for k, v in symbols_to_replace.items():
            s = s.replace(k, v)
    return s


In [ ]:
#PARAMETER_NAME

In [7]:
query = sql.SQL('''select distinct parameter_name_raw
                    from {}.{} 
                    where "loinc_code(T-luhend)" is null and parameter_name_raw is not null;
                ''').format(sql.Identifier('analysiscleaning'), sql.Identifier('parameter_name_to_t_lyhend_mapping'))
conn = pg2.connect(database_connection_string)
cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()
conn.commit()
cur.close()
#saame tabeli parmeter_name, parameter_code ja countiga
#hakkame puhastama parameter_name

In [8]:
rows

[('U-Bil:',),
 ('Nm-A(H1N1)',),
 ('vasksulfaat 2%',),
 ('f35 IgE kartul',),
 ('95 Tsefinaas (b-laktamaas)',),
 ('4 Atuupilised epiteelirakud',),
 ('Va-My(Seente kulv (tupest))',),
 ('TMP+SMX(TMP+SMX)',),
 ('Tsutomegaloviiruse vastane IgM seerumis*',),
 ('<ANONYM id="13" type="per" morph="_Y_ ?"/>(b) IgG',),
 ('Heinasegu',),
 ('Br-Mycobacterium-m',),
 ('Puukentsefaliidi viiruse vastase IgG hulk',),
 ('B-Kepptuumsed neutrofiilid %(Kepptuumsete neutrofiilide suhtarv)',),
 ('5 Aluseline fosfataas seerumis',),
 ('<ANONYM id="8" type="per" morph="_H_ sg n"/> Gap(K)',),
 ('Titiini ja SOX1 vastane IgG seerumis/plasmas',),
 ('Streptococcus B antigeen liikvoris',),
 ('ABO veregrupp (otsene)',),
 ('Abs. arv Neutrofiilid',),
 ('S-CRV',),
 ('P-CA 15-3',),
 ('Gamma-globuliin',),
 ('Muu leid(Muu leid (mida pole sademe mikroskoopias loetletud) vabateksti kujul)',),
 ('SSA/<ANONYM id="7" type="per" morph="_Y_ ?"/> IgG QN',),
 ('P-Gluc 120 min (post 75g Gluc PO)',),
 ('betta-2-mikroglobuliin',),
 ('F-Sa

In [9]:
i= 0
cols = ['dirty_code', 'clean_code']
tbl = pd.DataFrame(columns = cols)
#rows = [['B-Plt(Trombotsüütide arv)']]
for row in rows:
    pn = row[0]
    pn2 = sanitize_string(pn)
    pn2 = normalization(pn2)
    #print(pn2)
    if re.search(r'\(.*\)', pn):
        par = get_content_parantheses(pn2, pn2)
        #print('par', par)
        #print(get_content_parantheses(row[0], row[0]))
        if par != pn:
            new_row = pd.DataFrame([[pn, par[0]]], columns=cols)
            tbl = tbl.append(new_row)
    else:
        if pn != pn2:
            new_row = pd.DataFrame([[pn, pn2]], columns=cols)
            tbl = tbl.append(new_row)
print(i)
#tbl

0


In [10]:
tbl

,dirty_code,clean_code
0,U-Bil:,u-bil:
0,Nm-A(H1N1),nm-a
0,f35 IgE kartul,f35 ige kartul
0,95 Tsefinaas (b-laktamaas),tsefinaas
0,4 Atuupilised epiteelirakud,atuupilised epiteelirakud
0,Va-My(Seente kulv (tupest)),va-my(seentekulv
0,TMP+SMX(TMP+SMX),tmp+smx
0,Tsutomegaloviiruse vastane IgM seerumis*,tsutomegaloviiruse vastane igm seerumis*
0,"<ANONYM id=""13"" type=""per"" morph=""_Y_ ?""/>(b) IgG","<anonymid=""13""type=""per""morph=""y?""/>"
0,Heinasegu,heinasegu


In [12]:
#insert_to_database(tbl, 'analysiscleaning.dirty_parameter_name_cleaned', conn)

In [23]:
#tbl.loc[(tbl['dirty_code']).lower() != (tbl['clean_code']).lower() ]
#tbl['dirty_code'].equals(tbl['clean_code'])
#tbl['dirty_code'].str.lower()

tbl.loc[(tbl['dirty_code']).str.lower() != (tbl['clean_code']).str.lower() ]

,dirty_code,clean_code
0,Nm-A(H1N1),nm-a
0,95 Tsefinaas (b-laktamaas),tsefinaas
0,4 Atuupilised epiteelirakud,atuupilised epiteelirakud
0,Va-My(Seente kulv (tupest)),va-my(seentekulv
0,TMP+SMX(TMP+SMX),tmp+smx
0,"<ANONYM id=""13"" type=""per"" morph=""_Y_ ?""/>(b) IgG","<anonymid=""13""type=""per""morph=""y?""/>"
0,B-Kepptuumsed neutrofiilid %(Kepptuumsete neut...,b-kepptuumsedneutrofiilid%
0,5 Aluseline fosfataas seerumis,aluseline fosfataas seerumis
0,"<ANONYM id=""8"" type=""per"" morph=""_H_ sg n""/> G...","<anonymid=""8""type=""per""morph=""hsgn""/>gap"
0,ABO veregrupp (otsene),aboveregrupp


In [240]:
#PARAMETER_CODE

In [347]:
query = sql.SQL('''select distinct parameter_code_raw, count(*) from {}.{}
    where parameter_code_raw not in
    --eelenv kood
      (select distinct e.parameter_code_raw
      from analysiscleaning.dirty_code_to_t_lyhed_mapping_all as a
          right join work.run_ac_html_201901171409_analysis_entry as e
              on lower(a.dirty_code) = lower(e.parameter_code_raw)
      where dirty_code is not null
      )
     and not (parameter_code_raw  ~ '^[0-9]*$')
    group by parameter_code_raw
    order by count desc;
    ''').format(sql.Identifier('work'), sql.Identifier('run_ac_html_201901171409_analysis_entry'))
conn = pg2.connect(database_connection_string)
cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()
conn.commit()
cur.close()
#saame tabeli parmeter_name, parameter_code ja countiga
#hakkame puhastama parameter_name

In [348]:
i= 0
cols = ['dirty_code', 'clean_code']
tbl = pd.DataFrame(columns = cols)
for row in rows:
    pn = row[0]
    pn2 = sanitize_string(pn)
    pn2 = normalization(pn2)
    if re.search(r'\(.*\)', pn):
        par = get_content_parantheses(pn2, pn2)
        #print(get_content_parantheses(row[0], row[0]))
        if par != pn:
            new_row = pd.DataFrame([[pn, par[0]]], columns=cols)
            tbl = tbl.append(new_row)
    else:
        print('dirty PN:', pn2)
        if pn != pn2:
            new_row = pd.DataFrame([[pn, pn2]], columns=cols)
            tbl = tbl.append(new_row)
print(i)
tbl

dirty PN: b-cbc-5diff
dirty PN: u-strip
dirty PN: ab-ph
dirty PN: b-cbc-5diff-nrbc-ret
dirty PN: mch
dirty PN: mchc
dirty PN: pdw
dirty PN: rdw-cv
dirty PN: b-gluc
dirty PN: ab-p50
dirty PN: ab-pco2
dirty PN: ab-po2
dirty PN: ap-hco3
dirty PN: ab-pht
dirty PN: ab-shuntt
dirty PN: p-aptt
dirty PN: kaalium arteriaalses plasmas
dirty PN: naatrium arteriaalses plasmas
dirty PN: glukoos arteriaalses veres
dirty PN: ph
dirty PN: c-reaktiivne valk plasmas
dirty PN: kreatiniin paastuplasmas
dirty PN: uurea paastuplasmas
dirty PN: hinnanguline glomerulaarfiltratsiooni kiirus
dirty PN: beb
dirty PN: beecf
dirty PN: ca++
dirty PN: c to2
dirty PN: hco3
dirty PN: k+
dirty PN: na+
dirty PN: pco2
dirty PN: po2
dirty PN: sbc
dirty PN: so2 c%
dirty PN: tco2
dirty PN: a-ado2
dirty PN: ri
dirty PN: ab-glu
dirty PN: fio2
dirty PN: patsiendi temperatuur
dirty PN: rdw-sd
dirty PN: b3-xmb-g
dirty PN: ab-lac
dirty PN: ig-ebakupsedgranulotsuudid
dirty PN: ig%-ebakups.granulotsuutide%
dirty PN: ab-gluc
dirty PN

dirty PN: s-acla igamg
dirty PN: s-cmv igm
dirty PN: s-egfr
dirty PN: s-t gondii igg
dirty PN: s-t gondii igm
dirty PN: s-t pallidum ab
dirty PN: suhkur glu
dirty PN: triglutseriidid paastuplasmas
dirty PN: trombotsutoos
dirty PN: troponiin i
dirty PN: tureotropiin ehk kilpnaaret stimuleeriv hormoon plasmas
dirty PN: u-creakreatiniinuriinis
dirty PN: u-granulaarsilindrid
dirty PN: u-prot/crevalgujakreatiniinisuhe
dirty PN: u-protvalkkvantitatiivselt
dirty PN: urobilinogeen ubg
dirty PN: urobilinogeen uriinis
dirty PN: u-tbc
dirty PN: u-tbc-bac
dirty PN: valk pro
dirty PN:  yersinia antikehad
dirty PN:  enteroviirus
dirty PN:  s,p-troponiint hs-stat
dirty PN: acinetobacterbaumanniidnart-pcrmeetodiltaisverest
dirty PN: ana/tuumavastaneiggseerumis
dirty PN: aspergillusfumigatusdnart-pcrmeetodiltasiverest
dirty PN: b-106
dirty PN: b1-cd4#
dirty PN: b1-cd4#/cd8#
dirty PN: b1-cd8#
dirty PN: b1-hcv-pcrq
dirty PN: baktereemia/fungeemiart-pcrmeetodil
dirty PN: b-hba1c
dirty PN: b-hcv rna qn
dir

dirty PN: ft3/vabatrijoodturoniinseerumis
dirty PN: gamma globuliin
dirty PN: gammaglutamuuli transferaas
dirty PN: gehgentamutsiin30
dirty PN: glukoos kapillaarveres
dirty PN: glukoos seljaajuvedelikus
dirty PN: glukoos uriinis %
dirty PN: granuleeritud silindrid
dirty PN: hba1c
dirty PN: hba1c %
dirty PN: hhv6 dna
dirty PN: histamiin
dirty PN: hiv1.ja2.tuubivastaneantikeha,p24antigeenseerumis
dirty PN: hsv1 dna
dirty PN: hsv2 dna
dirty PN: hu igg
dirty PN: ig%-ebakupsetegranulotsuutidesuhtarv
dirty PN: igg1
dirty PN: igg2
dirty PN: igg3
dirty PN: igg4
dirty PN: inimese immuunpuudulikkuse viiruse 1. ja 2. tuubi vastased antikehad, p24 antigeen seerumis
dirty PN: isophorondiamin0,5%
dirty PN: kaaliumdikromaat0,50%
dirty PN: kaaliumditsuanoauraat0,002%
dirty PN: kaalium kapillaarveres
dirty PN: kadmiumsulfaat2%
dirty PN: kaltsium, ioniseeritud, seerumis
dirty PN: kaltsium paastuplasmas
dirty PN: kartsinoembruonaalne antigeen seerumis
dirty PN: kassikarv
dirty PN: kasvaja-antigeen ca 125

,dirty_code,clean_code
0,B-CBC-5Diff,b-cbc-5diff
0,U-Strip,u-strip
0,aB-pH,ab-ph
0,B-CBC-5DIFF-NRBC-RET,b-cbc-5diff-nrbc-ret
0,MCH,mch
0,MCHC,mchc
0,PDW,pdw
0,RDW-CV,rdw-cv
0,B-Gluc,b-gluc
0,aB-p50,ab-p50


In [349]:
db_operations.insert_to_database(tbl, 'analysiscleaning.dirty_code_to_t_lyhed_mapping_all_clean', conn)

In [120]:
s = 'kakskabsoluutarv'
if s.find('absoluutarv'):
    s = re.sub('absoluutarv', 'arv', s)
s

'kakskarv'

In [144]:
s = 'hjh abs())lklkl'
if s.find('abs'):
    s = re.sub('abs(.)?', 'arv', s)
s

'hjh arv))lklkl'

In [77]:
analysis_percent_absolut_values('bbb', 'aaaprotsent')

('bbb%', 'aaasuhtarv')